In [ ]:
from distributed import Scheduler
from tornado.ioloop import IOLoop
from threading import Thread

loop = IOLoop.current()
t = Thread(target=loop.start, daemon=True)
t.start()

s = Scheduler(loop=loop)
s.start('tcp://:8786')   # Listen on TCP port 8786

In [15]:
from dask.distributed import Client
client = Client()  # set up local cluster on your laptop
client

Client Scheduler: tcp://127.0.0.1:35011 Dashboard: http://127.0.0.1:41763,Cluster Workers: 4 Cores: 4 Memory: 4.14 GB


In [ ]:
from distributed import Client
client = Client('127.0.0.1:8786')

In [10]:
from time import sleep
from random import random

def inc(x):
    from random import random
    sleep(random() * 2)
    return x + 1

def double(x):
    from random import random
    sleep(random())
    return 2 * x


In [11]:
from queue import Queue
input_q = Queue()
remote_q = client.scatter(input_q)
inc_q = client.map(inc, remote_q)
double_q = client.map(double, inc_q)

In [ ]:
result_q = client.gather(double_q)

In [ ]:
result_q.qsize()

In [ ]:
input_q.put(10)
result_q.get()

In [ ]:
def load_data(q):
    i = 0
    while True:
        q.put(i)
        sleep(random())
        i += 1

from threading import Thread
load_thread = Thread(target=load_data, args=(input_q,))
load_thread.start()

In [ ]:
 while True:
        item = result_q.get()
        print(item)

In [ ]:
from queue import Queue
from threading import Thread

def multiplex(n, q, **kwargs):
    """ Convert one queue into several equivalent Queues

    >>> q1, q2, q3 = multiplex(3, in_q)
    """
    out_queues = [Queue(**kwargs) for i in range(n)]
    def f():
        while True:
            x = q.get()
            for out_q in out_queues:
                out_q.put(x)
    t = Thread(target=f)
    t.daemon = True
    t.start()
    return out_queues

def push(in_q, out_q):
    while True:
        x = in_q.get()
        out_q.put(x)

def merge(*in_qs, **kwargs):
    """ Merge multiple queues together

    >>> out_q = merge(q1, q2, q3)
    """
    out_q = Queue(**kwargs)
    threads = [Thread(target=push, args=(q, out_q)) for q in in_qs]
    for t in threads:
        t.daemon = True
        t.start()
    return out_q
